In [ ]:
%matplotlib inline
from sklearn.externals.joblib import Memory
from sklearn.datasets import load_svmlight_file
import numpy as np
from numpy import *
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False
mem = Memory("./mycache")

def Gw_NAG(w,x,y,size,b,vw,alpha,C,r):
    G=[0 for i in range(123)]
    L=[0 for i in range(size)]
    g=[0 for i in range(123)]
    sumG=0
    l=0
    x_temp=x*w
    for i in range(size):
        if(1-(y[i,0])*((x_temp)[i,0])-b>=0):
            for j in range(123):
                G[j]=G[j]-y[i,0]*x[i,j]
            L[i]=1-(y[i,0])*((x_temp)[i,0])-b
    for i in range(size):
        l=l+L[i]
    l=l*0.1
    for i in range(14):
        l=l+(w[i,0]**2)/2
    g=np.mat(G)
    g=g.T
    dw=w+C*g-r*vw
    vw=r*vw+alpha*dw
    w=w-vw
    return w

def Gb_NAG(w,x,y,size,b,vb,alpha,C,r):
    G=0
    g=[0 for i in range(123)]
    l=0
    x_temp=x*w
    for i in range(size):
        if(1-(y[i,0])*((x_temp)[i,0])-b>=0):
            G=G-y[i,0]
    db=C*G-r*vb
    vb=r*vb+alpha*db
    b=b-vb
    return b

def Gw_RMSProp(w,x,y,size,b,vw,alpha,C,r):
    G=[0 for i in range(123)]
    L=[0 for i in range(size)]
    g=[0 for i in range(123)]
    sumG=0
    l=0
    x_temp=x*w
    for i in range(size):
        if(1-(y[i,0])*((x_temp)[i,0])-b>=0):
            for j in range(123):
                G[j]=G[j]-y[i,0]*x[i,j]
            L[i]=1-(y[i,0])*((x_temp)[i,0])-b
    for i in range(size):
        l=l+L[i]
    l=l*0.1
    for i in range(14):
        l=l+(w[i,0]**2)/2
    g=np.mat(G)
    g=g.T
    dw=w+C*g
    vw=r*vw+(1-r)*np.multiply(dw,dw)
    w=w-np.multiply(alpha/(sqrt(vw+(1e-8))),dw)
    return w

def Gb_RMSProp(w,x,y,size,b,vb,alpha,C,r):
    G=0
    g=[0 for i in range(123)]
    l=0
    x_temp=x*w
    for i in range(size):
        if(1-(y[i,0])*((x_temp)[i,0])-b>=0):
            G=G-y[i,0]
    db=C*G
    vb=r*vb+(1-r)*np.multiply(db,db)
    b=b-np.multiply(alpha/(sqrt(vb+(1e-8))),db)
    return b

def Gw_AdaDelta(w,x,y,size,b,vw,C,r,tw):
    G=[0 for i in range(123)]
    L=[0 for i in range(size)]
    g=[0 for i in range(123)]
    sumG=0
    l=0
    x_temp=x*w
    for i in range(size):
        if(1-(y[i,0])*((x_temp)[i,0])-b>=0):
            for j in range(123):
                G[j]=G[j]-y[i,0]*x[i,j]
            L[i]=1-(y[i,0])*((x_temp)[i,0])-b
    for i in range(size):
        l=l+L[i]
    l=l*0.1
    for i in range(14):
        l=l+(w[i,0]**2)/2
    g=np.mat(G)
    g=g.T
    dw=w+C*g  
    vw=r*vw+(1-r)*np.multiply(dw,dw)
    Dw=np.multiply(-(sqrt(tw+(1e-8))/sqrt(vw+(1e-8))),dw)
    w=w+Dw
    tw=r*tw+np.multiply((1-r)*Dw,Dw)
    return w

def Gb_AdaDelta(w,x,y,size,b,vb,C,r,tb):
    G=0
    g=[0 for i in range(123)]
    l=0
    x_temp=x*w
    for i in range(size):
        if(1-(y[i,0])*((x_temp)[i,0])-b>=0):
            G=G-y[i,0]
    db=C*G
    vb=r*vb+(1-r)*np.multiply(db,db)
    Db=np.multiply(-(sqrt(tb+(1e-8))/sqrt(vb+(1e-8))),db)  
    b=b+Db
    tb=r*tb+np.multiply((1-r)*Db,Db)
    return b

def Gw_Adam(w,x,y,size,b,vw,alpha,C,r,mw,i):
    G=[0 for i in range(123)]
    L=[0 for i in range(size)]
    g=[0 for i in range(123)]
    beta=0.9
    sumG=0
    l=0
    x_temp=x*w
    for i in range(size):
        if(1-(y[i,0])*((x_temp)[i,0])-b>=0):
            for j in range(123):
                G[j]=G[j]-y[i,0]*x[i,j]
            L[i]=1-(y[i,0])*((x_temp)[i,0])-b
    for i in range(size):
        l=l+L[i]
    l=l*0.1
    for i in range(14):
        l=l+(w[i,0]**2)/2
    g=np.mat(G)
    g=g.T
    dw=w+C*g
    mw=beta*mw+(1-beta)*dw
    vw=r*vw+(1-r)*multiply(dw,dw)
    w=w-(alpha*(sqrt(1-r**(i+1)))/(1-beta**(i+1)))*(mw)/(sqrt(vw+(1e-8)))
    return w

def Gb_Adam(w,x,y,size,b,vb,alpha,C,r,mb,i):
    G=0
    g=[0 for i in range(123)]
    l=0
    beta=0.9
    x_temp=x*w
    for i in range(size):
        if(1-(y[i,0])*((x_temp)[i,0])-b>=0):
            G=G-y[i,0]
    db=C*G
    mb=beta*mb+(1-beta)*db
    vb=r*vb+(1-r)*multiply(db,db)
    b=b-(alpha*(sqrt(1-r**(i+1)))/(1-beta**(i+1)))*(mb)/(sqrt(vb+(1e-8)))
    return b


def validation(w,x,y,size,b):
    L=[0 for i in range(size)]
    l=0
    x_temp=x*w
    for i in range(size):
        if(1-(y[i,0])*((x_temp)[i,0]-b)>=0):
            L[i]=1-(y[i,0])*((x_temp)[i,0])
    for i in range(size):
        l=l+L[i]
    l=l*0.1
    for i in range(123):
        l=l+(w[i,0]**2)/2
    return l/size*10


@mem.cache
def get_data(a):
    data = load_svmlight_file(a,n_features=123)
    return data[0], data[1]

X_train, Y_train = get_data("G:\\Users\\qqqqqq1997520\\Desktop\\a9a.txt")
X_vali,Y_vali=get_data("G:\\Users\\qqqqqq1997520\\Desktop\\a9a_t.txt")
Xtrain=X_train.todense()
Xvali=X_vali.todense()
Xtrain=np.transpose(Xtrain)
Xtrain=Xtrain.T
Ytrain=np.mat(Y_train).T
Yvali=np.mat(Y_train).T

w_NAG=[0 for i in range(123)]
w_NAG=np.mat(w_NAG).T
w_RMSProp=w_NAG
w_AdaDelta=w_NAG
w_Adam=w_NAG

mw=[0 for i in range(123)]
mw=np.mat(mw).T
mb=0

sumT=0
C=0.1
alpha_NAG=0.002
alpha_RMSProp=0.001
alpha_Adam=0.02

r=0.9
vw_NAG=[0 for i in range(123)]
vw_NAG=np.mat(w_NAG).T
vb_NAG=0
vw_RMSProp=vw_NAG
vb_RMSProp=0
vw_AdaDelta=vw_NAG
vb_AdaDelta=0
vw_Adam=vw_NAG
vb_Adam=0

tw=[0 for i in range(123)]
tw=np.mat(tw).T
tb=0

LV_NAG=[0 for i in range(300)]
LV_RMSProp=[0 for i in range(300)]
LV_AdaDelta=[0 for i in range(300)]
LV_Adam=[0 for i in range(300)]

b_NAG=0
b_RMSProp=0
b_AdaDelta=0
b_Adam=0

x_train=[0 for i in range(640)]
y_train=[0 for i in range(640)]
for i in range(300):
    for j in range(640):
        a=random.randint(0,32560)
        x_train[j]=X_train[a];
        y_train[j]=Y_train[a];
    w_NAG=Gw_NAG(w_NAG,Xtrain,Ytrain,640,b_NAG,vw_NAG,alpha_NAG,C,r)
    b_NAG=Gb_NAG(w_NAG,Xtrain,Ytrain,640,b_NAG,vb_NAG,alpha_NAG,C,r)
    w_RMSProp=Gw_RMSProp(w_RMSProp,Xtrain,Ytrain,640,b_RMSProp,vw_RMSProp,alpha_RMSProp,C,0.995)
    b_RMSProp=Gb_RMSProp(w_RMSProp,Xtrain,Ytrain,640,b_RMSProp,vb_RMSProp,alpha_RMSProp,C,0.995)
    w_AdaDelta=Gw_AdaDelta(w_AdaDelta,Xtrain,Ytrain,640,b_AdaDelta,vw_AdaDelta,C,0.995,tw)
    b_AdaDelta=Gb_AdaDelta(w_AdaDelta,Xtrain,Ytrain,640,b_AdaDelta,vb_AdaDelta,C,0.995,tb)
    w_Adam=Gw_Adam(w_Adam,Xtrain,Ytrain,640,b_Adam,vw_Adam,alpha_Adam,C,0.995,mw,i)
    b_Adam=Gb_Adam(w_Adam,Xtrain,Ytrain,640,b_Adam,vb_Adam,alpha_Adam,C,0.995,mb,i)
    LV_NAG[i]=validation(w_NAG,Xvali,Yvali,16281,b_NAG)
    LV_RMSProp[i]=validation(w_RMSProp,Xvali,Yvali,16281,b_RMSProp)
    LV_AdaDelta[i]=validation(w_AdaDelta,Xvali,Yvali,16281,b_AdaDelta)
    LV_Adam[i]=validation(w_Adam,Xvali,Yvali,16281,b_Adam)

x=[i for i in range(300)]
plt.figure(1)
plt.subplot(211)
plt.plot(x,LV_NAG)
plt.plot(x,LV_RMSProp)
plt.plot(x,LV_AdaDelta)
plt.plot(x,LV_Adam)